# Projeto final de Data Science da Infinity  
> aluno: Walter Alves da Silva  
>link do dataset https://www.kaggle.com/datasets/mohammadtalib786/retail-sales-dataset

## import de bibilhotecas e dados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('retail_sales_dataset.csv')
df.head()

## limpeza de dados

### limpesa de dados desnescesarios

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.drop(['Transaction ID','Customer ID'], axis=1, inplace=True)

In [ ]:
df.info()

### engenharia de caracteristicas

#### Convertendo generos para binario

In [ ]:
df['Gender'].unique()

In [ ]:
gender = {'Male': 1,'Female': 0}
df.Gender = [gender[item] for item in df.Gender]
df.head()

#### convertendo a coluna Data para o formato data e depois para dia do ano 

In [ ]:
df['Date'].unique()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [ ]:
df.info()

In [ ]:
df["Date"] = df['Date'].apply(lambda x:x.dayofyear)
df.head()

considerando que se trata de dataset de compras, é de se esperar um padrão de repetição anual,  
por isso decidi converter a data para dia do ano, outro fator que me fez tomar a decisão  
é o fato de que todos os dados são do mesmo ano.

### regularizando variaveis categoricas

In [ ]:
from sklearn.preprocessing import OneHotEncoder

#Extract categorical columns from the dataframe
#Here we extract the columns with object datatype as they are the categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

#Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(df[categorical_columns])

#Create a DataFrame with the one-hot encoded columns
#We use get_feature_names_out() to get the column names for the encoded data
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate the one-hot encoded dataframe with the original dataframe
df = pd.concat([df, one_hot_df], axis=1)

# Drop the original categorical columns
df = df.drop(categorical_columns, axis=1)

df


usando o OneHotEncoder para separar as categorias de produtos em colunas binarias, dessa forma posso usar para treinar um modelo  
fonte: https://www.geeksforgeeks.org/ml-one-hot-encoding/

## analise exploratoria

### funçoes para plot

In [ ]:
def grafico_pizza(agrupadora:str, agrupada:str, nomes: list, titulo:str):
    """tem a função de criar um grafico pizza

    Args:
        agrupadora (str): coluna que vai ser as categorias
        agrupada (str): valores da categoria
        nomes (list): labels das categorias
        titulo (str): titulo do grafico
    """
    df_por_genero = df[[agrupadora, agrupada]]
    df_por_genero = df_por_genero.groupby(agrupadora, as_index=False).sum()

    # Extraindo as séries de categoria e valor
    categorias = df_por_genero[agrupadora].to_numpy()
    valores = df_por_genero[agrupada].to_numpy()

    # Criando o gráfico de pizza
    plt.figure(figsize=(2, 2))  # Definindo o tamanho do gráfico
    plt.pie(
        valores, labels=nomes, autopct="%1.1f%%", startangle=90
    )  # Criando o gráfico
    plt.title(titulo)  # Título do gráfico
    plt.axis("equal")  # Garantindo que o gráfico seja circular
    # Mostrando o gráfico
    plt.show()

In [ ]:
def grafico_linha(x:str,y:str,titulo:str):
    """_summary_

    Args:
        x (str): eixo X
        y (str):eixo Y
        titulo (str): titulo do grafico
    """
    df_graf = df[[x,y]]
    df_graf = df_graf.groupby(x,as_index=False).sum()

    x = df_graf[x]
    y = df_graf[y]

    plt.figure(figsize=(15, 6))  
    plt.plot(x, y, linestyle='-') 
    plt.title(titulo)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.grid(True)

    plt.xticks(rotation=45) 

    plt.tight_layout()
    plt.show()

### grafico valor Total X Dia

In [ ]:
df_por_data = df[['Date','Total Amount']]
df_por_data = df_por_data.groupby('Date',as_index=False).sum()
df_por_data.head()


In [ ]:
# Extraindo as séries de data e venda
datas = df_por_data["Date"]
vendas = df_por_data["Total Amount"]

# Criando o gráfico de linhas
plt.figure(figsize=(15, 6))  # Definindo o tamanho do gráfico
plt.plot(datas, vendas, linestyle='-')  # Criando a linha e definindo estilo
plt.title("Vendas por Dia do Ano")  # Título do gráfico
plt.xlabel("genero")  # Rótulo do eixo X
plt.ylabel("Valor da Venda")  # Rótulo do eixo Y
plt.grid(True)  # Exibindo grid no gráfico

# Formatando as datas no eixo X
plt.xticks(rotation=45)  # Rotacionando os rótulos das datas para melhor visualização

# Mostrando o gráfico
plt.tight_layout()
plt.show()

não é notado um padrão de compra, provavelmente isso se da ao fato de ser um dataset ficticio

### venda por genero

In [ ]:
df_por_genero = df[['Gender','Total Amount']]
df_por_genero = df_por_genero.groupby('Gender',as_index=False).sum()
df_por_genero.head()

# Extraindo as séries de categoria e valor
categorias = df_por_genero["Gender"].to_numpy()
valores = df_por_genero["Total Amount"].to_numpy()

# Criando o gráfico de pizza
plt.figure(figsize=(2, 2))  # Definindo o tamanho do gráfico
plt.pie(valores, labels=['feminino','masculino'], autopct="%1.1f%%", startangle=90)  # Criando o gráfico
plt.title("Distribuição por genero")  # Título do gráfico
plt.axis('equal')  # Garantindo que o gráfico seja circular

# Mostrando o gráfico
plt.show()

as vendas variam em 2,2% entre os generos, ao meu ver não é uma diferença consideravel, e o fato de ser um dataset artificial pode ter influenciado

### grafico idade X venda

In [ ]:
df_por_data = df[['Age','Total Amount']]
df_por_data = df_por_data.groupby('Age',as_index=False).sum()
df_por_data.head()

x = df_por_data["Age"]
y = df_por_data["Total Amount"]

plt.figure(figsize=(15, 6))  
plt.plot(x, y, linestyle='-') 
plt.title("Vendas por Idade")
plt.xlabel("Idade")
plt.ylabel("Valor da Venda")
plt.grid(True)

plt.xticks(rotation=45) 

plt.tight_layout()
plt.show()

novamente nenhum padrão, esse dataset é muito sintetico, sera possivel treinar um modelo?'

### genero por categoria

In [ ]:
df.columns

In [ ]:
df_por_data = df[
    [
        "Gender",
        "Total Amount",
        "Product Category_Beauty",
        "Product Category_Clothing",
        "Product Category_Electronics",
    ]
]
df_por_data = df_por_data.groupby("Gender", as_index=False).sum()
df_por_data

In [ ]:
grafico_pizza('Gender','Product Category_Beauty',['feminino','masculino'], 'Genero, produto de beleza')

notorio que as mulheres compram mais produtos de beleza, como esperado

In [ ]:
grafico_pizza('Gender','Product Category_Clothing',['feminino','masculino'], 'Genero x roupa')

In [ ]:
grafico_pizza('Gender','Product Category_Electronics',['feminino','masculino'], 'Genero X eletronico')

### idade X categoria

In [ ]:
for categoria in ['Product Category_Beauty', 'Product Category_Clothing','Product Category_Electronics']:
    grafico_linha('Age',categoria,f'Idade x {categoria}')

nenhum padrão observado

### correlação

In [ ]:
import seaborn as sns

In [ ]:
corr_matrix = df.corr()
corr_matrix

In [ ]:
sns.heatmap(corr_matrix, annot = True, cmap = 'coolwarm', vmin = -1, vmax = 1)
plt.show()

### resumo

considerando as informaçoes extraidads dos graficos notase que não é possivel obter uma corelação entre os valores   
exceto genetro por venda de produto de beleza.

## modelo Machine Learning

### import e escalonamento

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [ ]:
x = df.drop(columns = [ 'Quantity', 'Price per Unit', 'Total Amount'])
y = df['Total Amount']

escalonador = StandardScaler()
x_escalonado = escalonador.fit_transform(x)

x_treino, x_teste, y_treino, y_teste = train_test_split(x_escalonado, y)

# ['Date', 'Gender', 'Age', 'Quantity', 'Price per Unit', 'Total Amount','Product Category_Beauty', 'Product Category_Clothing','Product Category_Electronics']

### regressor linear

In [ ]:
regressor = LinearRegression()
regressor.fit(x_treino, y_treino)

In [ ]:
previsao = regressor.predict(x_teste)
previsao

In [ ]:
mse = mean_squared_error(y_teste, previsao)
mse

### regressor Perceptron

In [ ]:
regressor = Perceptron()
regressor.fit(x_treino, y_treino)

In [ ]:
previsao = regressor.predict(x_teste)
previsao

In [ ]:
mse = mean_squared_error(y_teste, previsao)
mse

### outro regressor

notase um erro muito grande, isso acontesse por que o dataset analisado foi gerado aleatoriamente, como observado pela anasile exploratoria,  
o machine learning tem como objetivo buscar padroes, e como no caso não ouve padrão, é de se esperar que o computador não consiga o seu objetivo.